In [1]:
# -*- coding: utf-8 -*-
# python3 example_cifar10_scnn.py
import sys
sys.path.append("..")
from importlib import import_module
import torch,time,os,random
import torch.nn as nn
import torch.nn.functional as F
import sys
sys.path.append("..")
import LIAF
from datasets.cifar10_dvs_simple import cifar10_DVS
from tensorboardX import SummaryWriter
from importlib import import_module
from LIAFnet.LIAFResNet import *
autocast = LIAF.autocast #统一autocast模式

################################ parameters ####################

for test in range(1):
    os.environ['CUDA_VISIBLE_DEVICES'] = "1"
    task = '32ESpretrain' + str(test)
    writer = SummaryWriter(comment=task)
    learning_rate = 3e-3
    batch_size  = 500
    num_epochs = 100
    timeWindows = 8
    names = 'dvs_cifar10'
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_path = '/data/CIFAR10-DVS-mini/train.mat' #r := raw string
    test_path =  '/data/CIFAR10-DVS-mini/test.mat' #TODO:input your oath

    train_dataset = cifar10_DVS(train_path,'r',timeWindows) #max= 25?
    test_dataset = cifar10_DVS(test_path,'r',timeWindows)

    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=batch_size,
                                               shuffle=True)

    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                               batch_size=batch_size,
                                               shuffle=False)

    TA.timeWindows = 8 #用了attention必须固定时间长度
    modules = import_module('LIAFnet.LIAFResNet_18')
    config  = modules.Config()
    workpath = os.path.abspath(os.getcwd())
    num_epochs = config.num_epochs
    accumulation = config.accumulation
    timeWindows = config.timeWindows
    config.cfgCnn = [2, 64, 7, True]
    config.dataSize = (42,42)
    config.actFun= LIAF.LIFactFun.apply
    epoch = 0
    bestacc = 0  # best test accuracy
    start_epoch = 0  # start from epoch 0 or last checkpoint epoch
    training_iter = 0
    snn = LIAFResNet(config)
    checkpoint = torch.load('./45.pkl', map_location=torch.device('cpu'))
    snn.load_state_dict(checkpoint)
    for p in snn.parameters():
        p.requires_grad=False #TODO
    snn.fc = nn.Linear(snn.cfgFc_[0],10)
    print("Total number of paramerters in networks is {}  ".format(sum(x.numel() for x in snn.parameters())))

    snn.to(device)
    criterion = nn.CrossEntropyLoss()

    optimizer = torch.optim.Adam(snn.parameters(),
                    lr=learning_rate,
                    weight_decay =1e-4)


    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(
                        optimizer, 
                        milestones=[20,40,60], 
                        gamma=0.1, 
                        last_epoch=-1)
    def val(optimizer,snn,test_loader,test_dataset,batch_size,epoch):
        print('===> evaluating models...')
        snn.eval()
        correct = 0
        total = 0
        predicted = 0
        with torch.no_grad():
            for batch_idx, (inputs, labels) in enumerate(test_loader):
                inputs = inputs.permute([0,1,4,2,3])
                if ((batch_idx+1)<=len(test_dataset)//batch_size):
                    optimizer.zero_grad()
                    outputs = snn(inputs.to(device))

                    _ , labels = labels.max(1)
                    _ , predicted = outputs.cpu().max(1)
                    total += float(labels.size(0))
                    correct += float(predicted.eq(labels).sum())
        acc = 100. * float(correct) / float(total)
        print('================')
        print('val acc:',acc , '%  epoch:',epoch)
        print('================')
        writer.add_scalar('acc', acc, epoch)
        return acc

    save_folder = './saved_model'

    for epoch in range(num_epochs):
        # 新增2：设置sampler的epoch，DistributedSampler需要这个来维持各个进程之间的相同随机数种子
        if epoch>1:
            for p in snn.parameters():
                p.requires_grad=True #TODO
        snn.train()
        running_loss = 0
        start_time = time.time() 
        print('===> training models...')
        correct = 0.0
        total = 0.0
        torch.cuda.empty_cache()
        for i, (images, labels) in enumerate(train_loader):
            images = images.permute([0,1,4,2,3])
            if ((i+1)<=len(train_dataset)//batch_size):
                with autocast():
                    outputs = snn(images.to(device)).cpu()
                    _,labels = labels.max(1)
                    _ , predict = outputs.max(1)
                    loss = criterion(outputs, labels)
                    correct += predict.eq(labels).sum()
                    total += float(predict.size(0))

                    loss /= accumulation
                    running_loss += loss.item()
                    loss.backward()

                if (i+1)%accumulation == 0:
                    optimizer.step()
                    snn.zero_grad()
                    optimizer.zero_grad()

                if (i+1)%(accumulation*18) == 0:
                    if not os.path.isdir(save_folder):
                        os.mkdir(save_folder)
                    train_acc =  100* correct / total
                    print('=====> Epoch [%d/%d], Step [%d/%d], Loss: %.5f , Acc: %.5f  '
                      %(epoch+start_epoch, num_epochs+start_epoch, i+1, len(train_dataset)//(batch_size),running_loss, train_acc))
                    writer.add_scalar('running Loss', running_loss, training_iter)
                    writer.add_scalar('running Acc', train_acc, training_iter)
                    correct = 0.0
                    total = 0.0
                    running_loss = 0
            training_iter +=1 
        torch.cuda.empty_cache()
        #evaluation
        acc = val(optimizer,snn,test_loader,test_dataset,batch_size,epoch)
        lr_scheduler.step()
        if not os.path.isdir(save_folder):
            os.mkdir(save_folder)
        if acc > bestacc:
            bestacc = acc
            print('===> Saving models...')


    torch.save(snn.state_dict(),'./'+save_folder+'/'+task+str(int(bestacc))+'.pkl')

use torch.cuda.amp.autocast for fusion prcision training
torch.Size([9000, 2, 42, 42, 8]) torch.Size([9000, 10])
torch.Size([1000, 2, 42, 42, 8]) torch.Size([1000, 10])
Total number of paramerters in networks is 11185379  
===> training models...


/home/lyh/anaconda3/envs/pytorch/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448255797/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


=====> Epoch [0/40], Step [18/18], Loss: 40.07527 , Acc: 18.34444  
===> evaluating models...
val acc: 11.3 %  epoch: 0
===> Saving models...
===> training models...
=====> Epoch [1/40], Step [18/18], Loss: 37.65362 , Acc: 25.38889  
===> evaluating models...
val acc: 24.7 %  epoch: 1
===> Saving models...
===> training models...
=====> Epoch [2/40], Step [18/18], Loss: 35.04057 , Acc: 29.63333  
===> evaluating models...
val acc: 11.3 %  epoch: 2
===> training models...
=====> Epoch [3/40], Step [18/18], Loss: 29.68761 , Acc: 40.14444  
===> evaluating models...
val acc: 20.1 %  epoch: 3
===> training models...
=====> Epoch [4/40], Step [18/18], Loss: 28.58351 , Acc: 43.90000  
===> evaluating models...
val acc: 15.4 %  epoch: 4
===> training models...
=====> Epoch [5/40], Step [18/18], Loss: 28.15687 , Acc: 43.65556  
===> evaluating models...
val acc: 15.3 %  epoch: 5
===> training models...
=====> Epoch [6/40], Step [18/18], Loss: 27.58492 , Acc: 45.12222  
===> evaluating models..